# Получение данных

В идеальной картине мира, с которой мы бы работали после того, как сервис будет готов, мы бы получали данные из базы фестиваля напрямую или с помощью API. Но в текущих абстоятельствах мы проявим чудеса выдумки,с мекалки и просто трудолюбия чтобы получить очень хорошие результаты и не упороться.

## Что мы имеем?

Так как мы будем делать проект на исторических данных фестиваля за конкретный год — он был очень яркий и показательный с точки зрения программы - у нас есть выгрузка всех транзакций из базы. Из нее мы можем вытащить дни, время, место, название, цену и т.д. Т.е. создать весь костяк данных.

Отдельно у нас есть подготовленные выгрузки с артистами, произведениями и их авторами с их привязками в конректные концерты.


In [1]:
import pandas as pd
import datetime

# Опишем пути к файлам
TRANSACTIONS_PATH = 'data_dirty/bd2022_06072022.xlsx'
ARTISTS_PATH = 'data_dirty/ArtistDetails.xlsx'
PROGRAMM_PATH = 'data_dirty/ProgrammDetails.xlsx'

SHOW_LONG = 45 #длительность концерта по умолчанию
TICKETS_STATE = True #состояние билетов по умолчанию



## Праметры фестиваля

Первым делом мы хотим формировать картину фестиваля. Извлечь данные о конертах и фестивальных днях. Но перед этим чуть чуть почистим выгрузки.

В файле с транзакциями много полезной информации, но так же есть и избыточные данные. В частности, операции возврата билетов. Их отличает значение Price < 0. 

In [2]:
transactions = pd.read_excel(TRANSACTIONS_PATH)

In [3]:
Vozvrat = transactions[transactions['Price'] < 0]
print(f'Всего операций возвратов в выгрузке: {len(Vozvrat)}')

#удалим операции возвратов, чтобы они не влияли на расчет средних значений (напрмиер цены билаов на этот концерт)
transactions = transactions.drop(transactions[transactions['Price'] < 0].index)

#Из полученного списка транзакций возьмем только колонки, которые относятся к мероприятиям и соберем список из уникальных значений
concerts = transactions[['ShowId','ShowName','ShowDate','HallName','Genre']].drop_duplicates()
concerts

Всего операций возвратов в выгрузке: 26


,ShowId,ShowName,ShowDate,HallName,Genre
0,100,100. Закрытие фестиваля,2022-07-03 20:15:00,Сад Вайнера — 2021 год,Симфонический оркестр
3,87,78. Сказка с оркестром. Русалочка,2022-07-03 12:30:00,ТЮЗ - Большой,Камерные программы
5,69,68. Шоу барабанщиков «Чувство ритма»,2022-07-02 21:00:00,Сад Вайнера — 2021 год,Камерные программы
7,49,48. Этно-шоу. Сотворение мира,2022-07-02 14:30:00,ТЮЗ - Большой,Народные инструменты
9,6,1. Симфонический рояль,2022-07-01 13:00:00,Камерный зал филармонии,Камерные программы
...,...,...,...,...,...
1804,24,16. Романтический кроссовер,2022-07-01 17:00:00,Дом музыки,Народные инструменты
1826,25,23. Романтическое путешествие,2022-07-01 19:15:00,Дом музыки,Камерный оркестр
1921,65,50. Хоровая романтика III,2022-07-02 15:00:00,Камерный театр,Хор
2145,91,82. Музыкальный момент,2022-07-03 13:15:00,ТЮЗ - Малый,Народные инструменты


In [4]:
#в названии концертов фестиваля Безумных дней указывается его порядковый номер. Разделим строку на номер и собственно название
qq = concerts['ShowName'].str.split('. ', n=1, expand = True)

concerts['ShowNum'] = pd.to_numeric(qq[0])
concerts['ShowName'] = qq[1]

# Назначим в качестве индекса номер концерта и отсортируем список.
concerts = concerts.set_index('ShowNum').sort_index()


In [5]:
#Добавим к концерту среднюю стоимость билета. Цена так-то фиксирована, но чтобы все было системно и нчиего не пропустить.
qq = pd.DataFrame(transactions.groupby('ShowName')['Price'].mean()).reset_index()
MeanPrice = pd.DataFrame({'ShowNum': pd.to_numeric(qq['ShowName'].str.split('. ', n=1, expand = True)[0]),
                          'Price': qq['Price']})
MeanPrice = MeanPrice.set_index('ShowNum').sort_index()

concerts = pd.merge(concerts, MeanPrice, how='left', left_index=True, right_index=True)


#в перечне транзакций длительность концерта, безусловно, не указывается. Но в конечном спсике она нам нужна.
#Пока мы не получим других данных, считаем, что все концерты дляться заявленные 45 минут
concerts['ShowLong'] = datetime.time(hour = 0, minute = SHOW_LONG)

#У каждого концерта должен быть параметр, который описывает наличие\отсутствие билетов
# вотсутствие билетов концерт не будет рассматриваться для построения маршрута
# В дальнейшем мы будем запрашивать актуальное состояние билетов из базы фестиваля
concerts['Tickets'] = TICKETS_STATE

# У каждого концерта должна быть ссылка на страницу, где его можно купить
concerts['link'] = ''


concerts

,ShowId,ShowName,ShowDate,HallName,Genre,Price,ShowLong,Tickets,link
ShowNum,,,,,,,,,
1,6,Симфонический рояль,2022-07-01 13:00:00,Камерный зал филармонии,Камерные программы,200.0,00:45:00,True,
2,22,Кармен и Пер Гюнт,2022-07-01 13:00:00,Дом музыки,Камерные программы,200.0,00:45:00,True,
3,10,Шопен. Соната и блестящий полонез,2022-07-01 13:30:00,Зал Маклецкого,Камерные программы,200.0,00:45:00,True,
4,14,Шуман и Берлиоз,2022-07-01 13:30:00,ТЮЗ - Большой,Симфонический оркестр,200.0,00:45:00,True,
5,26,Хоровая романтика I,2022-07-01 13:30:00,Камерный театр,Хор,200.0,00:45:00,True,
...,...,...,...,...,...,...,...,...,...
96,80,Венеция и Неаполь,2022-07-03 18:00:00,Камерный зал филармонии,Камерные программы,200.0,00:45:00,True,
97,93,Два гобоя и английский рожок,2022-07-03 18:00:00,ТЮЗ - Малый,Камерные программы,200.0,00:45:00,True,
98,90,Фламенко в ритме сердца,2022-07-03 18:45:00,ТЮЗ - Большой,Танец,200.0,00:45:00,True,


## Сетка залов

После извлечем список концертных площадок. На его основе чуть позже мы будем строить сетку переходов между залами. Этот параметр будет учитываться при составлении реалистичных маршрутов.

In [28]:
concerts['HallName'].value_counts()

HallName
Большой зал филармонии     16
Зал Маклецкого             15
Камерный зал филармонии    14
Дом музыки                 14
ТЮЗ - Большой              14
Камерный театр             12
ТЮЗ - Малый                12
Сад Вайнера — 2021 год      3
Name: count, dtype: int64

In [36]:
# Сохраним список залов и создадим од него структуру, которую будет чем дальше наполнить
HallList = pd.DataFrame(concerts['HallName'].value_counts())
HallList.sort_index(inplace=True)
HallList.index.name = 'HallName'
HallList['Adress'] = ''
HallList['latitude'] = 0
HallList['longitude'] = 0
HallList


,count,Adress,latitude,longitude
HallName,,,,
Большой зал филармонии,16,,0,0
Дом музыки,14,,0,0
Зал Маклецкого,15,,0,0
Камерный зал филармонии,14,,0,0
Камерный театр,12,,0,0
Сад Вайнера — 2021 год,3,,0,0
ТЮЗ - Большой,14,,0,0
ТЮЗ - Малый,12,,0,0


In [37]:
CLEAR_DATA_PATH = 'data_clean/'
HallList.to_excel(CLEAR_DATA_PATH + 'HallList.xlsx')


In [21]:
# На пересечении строк и столбцов мы после укажем время, которое потребуется клиенту чтобы перейти из зала А в зал Б
HallsTime = pd.DataFrame(0, index=HallList.index, columns=HallList.index)
HallsTime 

HallName,Большой зал филармонии,Дом музыки,Зал Маклецкого,Камерный зал филармонии,Камерный театр,Сад Вайнера — 2021 год,ТЮЗ - Большой,ТЮЗ - Малый
HallName,,,,,,,,
Большой зал филармонии,0,0,0,0,0,0,0,0
Дом музыки,0,0,0,0,0,0,0,0
Зал Маклецкого,0,0,0,0,0,0,0,0
Камерный зал филармонии,0,0,0,0,0,0,0,0
Камерный театр,0,0,0,0,0,0,0,0
Сад Вайнера — 2021 год,0,0,0,0,0,0,0,0
ТЮЗ - Большой,0,0,0,0,0,0,0,0
ТЮЗ - Малый,0,0,0,0,0,0,0,0


## Дополнительные данные о концерта

Подгрузим данные об артистах, программах концерта и авторах произведений.

### Программы концертов

In [22]:
#Получим файл с программой концертов
ShowDetails = pd.read_excel(PROGRAMM_PATH).set_index('ShowNum')

#Разделим столбец со строкой на автора и название произведения
ShowDetails = ShowDetails['Programm'].str.split('.', n=1, expand = True)
ShowDetails.columns = ['Author','Programm']

#Добавим столбец, в котором будем писать True если редакция проекта его "рекомендует"
ShowDetails['Spetial'] = None

ShowDetails



,Author,Programm,Spetial
ShowNum,,,
1,Шуман,"Арабески, ор. 18",None
1,Шуман,"Симфонические этюды, op. 13",None
2,Бизе—Ито,Фантазия на темы из оперы «Кармен»,None
2,Шуман,"«Альбом для юношества», op. 68 (№№1-10)",None
2,Григ—Эванс,Сюита из музыки к драме Ибсена «Пер Гюнт»,None
...,...,...,...
100,Шуберт,«Аве Мария»,None
100,Шуман,Адажио и аллегро для виолончели и фортепиано,None
100,Шопен,Концерт № 2 для фортепиано с оркестром (III ч...,None


In [23]:
#посчитаем уникальных композиторов в программе каждого концерта
aaa = ShowDetails['Author'].reset_index().drop_duplicates().groupby('ShowNum').count()
concerts = concerts.merge(aaa, how='left', left_index=True, right_index=True)

#посчитаем уникальных произведений в программе каждого концерта
aaa = ShowDetails['Programm'].groupby('ShowNum').count()
concerts = concerts.merge(aaa, how='left', left_index=True, right_index=True)

concerts

,ShowId,ShowName,ShowDate,HallName,Genre,Price,ShowLong,Tickets,link,Author,Programm
ShowNum,,,,,,,,,,,
1,6,Симфонический рояль,2022-07-01 13:00:00,Камерный зал филармонии,Камерные программы,200.0,00:45:00,True,,1,2
2,22,Кармен и Пер Гюнт,2022-07-01 13:00:00,Дом музыки,Камерные программы,200.0,00:45:00,True,,6,6
3,10,Шопен. Соната и блестящий полонез,2022-07-01 13:30:00,Зал Маклецкого,Камерные программы,200.0,00:45:00,True,,2,3
4,14,Шуман и Берлиоз,2022-07-01 13:30:00,ТЮЗ - Большой,Симфонический оркестр,200.0,00:45:00,True,,2,2
5,26,Хоровая романтика I,2022-07-01 13:30:00,Камерный театр,Хор,200.0,00:45:00,True,,4,4
...,...,...,...,...,...,...,...,...,...,...,...
96,80,Венеция и Неаполь,2022-07-03 18:00:00,Камерный зал филармонии,Камерные программы,200.0,00:45:00,True,,1,5
97,93,Два гобоя и английский рожок,2022-07-03 18:00:00,ТЮЗ - Малый,Камерные программы,200.0,00:45:00,True,,3,7
98,90,Фламенко в ритме сердца,2022-07-03 18:45:00,ТЮЗ - Большой,Танец,200.0,00:45:00,True,,1,1


### Артисты

In [24]:
#Получим файл с транзакциями
ArtistDetails = pd.read_excel(ARTISTS_PATH).set_index('ShowNum')
ArtistDetails


,Artists
ShowNum,
1,Константин Емельянов (фортепиано)
2,Российский квартет саксофонистов
3,Дмитрий Прокофьев (виолончель)
3,Рэм Урасин (фортепиано)
4,Уральский молодёжный симфонический оркестр
...,...
100,Екатерина Мечетина (фортепиано)
100,Юрий Фаворин (фортепиано)
100,Альбина Латипова (сопрано)


In [25]:
# Получим список уникальных артистов, которые выступают на фестивале
ArtistList = ArtistDetails['Artists'].drop_duplicates().sort_values()
ArtistList = ArtistList.reset_index().drop('ShowNum', axis=1)

#Добавим столбец, в котором будем писать True если редакция проекта его "рекомендует"
ArtistList['Spetial'] = None

ArtistList

,Artists,Spetial
0,Compromise-quartet,None
1,Shepherd trio,None
2,«ТанцТеатр»,None
3,Александр Борисов (художественное слово),None
4,Александр Зинченко (скрипка),None
...,...,...
69,"Хироко Иноуэ (орган), Япония",None
70,Шоу барабанщиков «Чувство ритма»,None
71,Юрий Нечаев (кларнет),None
72,Юрий Фаворин (фортепиано),None


# Параметры всего и вся

Статистические и динамичические параметры по фестивалю и отдельным его аспектам нас пригодятся еще множество раз, потому проведем некоторую фиксацию наших достижений.

### Статистика по артистам

Для артистов нам тоже будет интересно оценить в какой доле концертов они задействованы. Ведь концерты бывают сольные, бывают ансамблевые. А бывает что человек очень талантливый и пролез много где.

In [26]:
ArtistDetails['Count'] = 1
ArtistStats = ArtistDetails.groupby('Artists').count().sort_values(by='Count', ascending=False)
ArtistStats['Part'] = ArtistStats['Count'] / len(ArtistDetails.index.drop_duplicates())

ArtistStats

,Count,Part
Artists,,
Екатерина Мечетина (фортепиано),7,0.07
Российский квартет саксофонистов,7,0.07
Рэм Урасин (фортепиано),6,0.06
Shepherd trio,6,0.06
Дирижёр — Дмитрий Лисс,6,0.06
...,...,...
Ли Илунь (тенор),1,0.01
Мария Анчугова (сопрано),1,0.01
Военный оркестр штаба Центрального военного округа,1,0.01


### Статситика по авторам

Часть собранной статистики нам просто покажется любопытной для общей сводки по фестивалю, а часть будет влиять при составлении маршрута. Для начала, посчитаем долю произведений автора во всем объеме программы фестиваля. Оцениваем чисто штуки (этого более чем достаточно)


In [27]:
AuthorStats = ShowDetails.groupby('Author').count().sort_values('Programm', ascending=False)
AuthorStats['Part'] = AuthorStats['Programm'] / AuthorStats['Programm'].sum()

AuthorStats

,Programm,Spetial,Part
Author,,,
Шуман,60,0,0.153453
Шопен,53,0,0.135550
Мендельсон,42,0,0.107417
Шуберт,31,0,0.079284
Бах И,21,0,0.053708
Лист,21,0,0.053708
Брамс,20,0,0.051151
_Прочее,16,0,0.040921
Бетховен,10,0,0.025575


### Статистика по покупателям

Не то чтобы эта метрика нам прям требовалась на этом этапе, но у нас же есть фаль транзакций! Значит, мы можем оценить, какое количество человек у нас вовлечено в фестиваль, сколько они покупают на нос. Что в той или иной мере влияет на граничные характеристики нашего продукта.

In [37]:
# Посчитаем количество уникальных клиентов, которые купили билеты на фестиваль
clients_count = len(transactions['ClientId'].unique())
print('Количество уникальных клиентов:', clients_count)

Количество уникальных клиентов: 4232


In [40]:
#Посчитаем количество билетов, купленных каждым клиентом
clients_top = transactions.groupby('ClientId').count().sort_values(by='Price', ascending=False)['ShowId'].head(10)
print('Количество билетов, купленных каждым клиентом (топ-10):', clients_top)

Количество билетов, купленных каждым клиентом (топ-10): ClientId
18269.0    198
37138.0    124
63099.0     87
62415.0     76
27013.0     61
51256.0     59
73464.0     58
96903.0     58
36226.0     54
62673.0     53
Name: ShowId, dtype: int64


In [49]:
#Покажем, какое количество билетов в срезе номеров концертов клиент тоже приобредл
transactions.groupby(['ClientId', 'ShowId']).count().sort_values(by='ShowName', ascending=False)['ShowName']


ClientId  ShowId
18269.0   69        23
97992.0   68        19
18269.0   100       18
          74        18
30080.0   85        14
                    ..
89657.0   41         1
          28         1
89653.0   85         1
89650.0   32         1
95737.0   32         1
Name: ShowName, Length: 12753, dtype: int64

In [ ]:
pd.read_excel(TRANSACTIONS_PATH)

,OpId,OpDate,WebOrderId,HallName,ShowDate,ShowName,ShowId,Genre,Fest,TicketType,TicketNum,ZoneName,RowNum,SeatNum,Price,ClientId,Pushkin
0,247,2022-04-21 00:07:12,228.0,Сад Вайнера — 2021 год,2022-07-03 20:15:00,100. Закрытие фестиваля,100,Симфонический оркестр,1,билет,100,Партер,11.0,19,500,64400.0,0
1,248,2022-04-21 00:07:12,228.0,Сад Вайнера — 2021 год,2022-07-03 20:15:00,100. Закрытие фестиваля,100,Симфонический оркестр,1,билет,100,Партер,11.0,18,500,64400.0,0
2,249,2022-04-21 00:07:12,228.0,Сад Вайнера — 2021 год,2022-07-03 20:15:00,100. Закрытие фестиваля,100,Симфонический оркестр,1,билет,100,Партер,11.0,17,500,64400.0,0
3,258,2022-04-21 00:08:25,239.0,ТЮЗ - Большой,2022-07-03 12:30:00,78. Сказка с оркестром. Русалочка,87,Камерные программы,1,билет,87,Партер,3.0,12,200,57763.0,0
4,259,2022-04-21 00:08:25,239.0,ТЮЗ - Большой,2022-07-03 12:30:00,78. Сказка с оркестром. Русалочка,87,Камерные программы,1,билет,87,Партер,3.0,13,200,57763.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36363,99237,2022-07-02 12:29:00.000,NaN,ТЮЗ - Большой,2022-07-02 12:30:00,41. Пляж на Лебедином озере,48,Камерные программы,1,билет,48,Партер,19.0,2,200,NaN,0
36364,99238,2022-07-02 12:29:00.000,NaN,ТЮЗ - Большой,2022-07-02 12:30:00,41. Пляж на Лебедином озере,48,Камерные программы,1,билет,48,Партер,19.0,3,200,NaN,0
36365,99239,2022-07-02 12:18:00.000,NaN,ТЮЗ - Большой,2022-07-02 12:30:00,41. Пляж на Лебедином озере,48,Камерные программы,1,билет,48,Партер,20.0,18,200,NaN,0
36366,99240,2022-07-02 12:18:00.000,NaN,ТЮЗ - Большой,2022-07-02 12:30:00,41. Пляж на Лебедином озере,48,Камерные программы,1,билет,48,Партер,20.0,17,200,NaN,0


### Итоговые параметры фестиваля

In [31]:
#Получим даты концертных дней
FestDays = pd.DataFrame({'Day': concerts['ShowDate'].dt.date.unique()})#.set_index('Day')


#Выпишем начало первого и последнего концерта каждого дня фестиваля
df = pd.DataFrame(
        {'Day': concerts['ShowDate'].dt.strftime('%Y-%m-%d'),
        'Time': concerts['ShowDate'].dt.strftime('%H:%M')}
    ).groupby('Day').agg(['min','max']).reset_index()
df.columns = df.columns.droplevel(0)
FestDays[['Start', 'Finish']] = df[['min','max']]


#Посчитаем у скольки концертов билеты еще есть в продаже
df = pd.DataFrame(
        {'Day': concerts['ShowDate'].dt.strftime('%Y-%m-%d'),
        'Tickets': concerts['Tickets']}
    ).groupby('Day').sum().reset_index()
FestDays['Tickets'] = df['Tickets']



#Отметим, сколько всего концертов в каждом дне
FestDays['NumShows'] = pd.DataFrame({'NumShows': concerts['ShowDate'].dt.strftime('%Y-%m-%d').value_counts().sort_index()}).reset_index()['NumShows']


FestDays = FestDays.set_index('Day')
FestDays


,Start,Finish,Tickets,NumShows
Day,,,,
2022-07-01,13:00,22:15,30,30
2022-07-02,10:00,22:15,39,39
2022-07-03,10:00,20:15,31,31


In [75]:
concerts_num = len(concerts)
fest_days = len(transactions['ShowDate'].unique())
halls_num = len(concerts['HallName'].unique())

In [76]:
print(f'''
      Дней фестиваля: {fest_days}
      Всего концертов: {concerts_num}
      Залов: {halls_num}
''')


      Дней фестиваля: 77
      Всего концертов: 100
      Залов: 8



## Сохраним чистые данные

Сохраним собранные и обогащенные данные в новые файлы, чтобы дальнешйие работы вести обращаясь к ним.

In [ ]:
CLEAR_DATA_PATH = 'data_clean/'
concerts.to_excel(CLEAR_DATA_PATH + 'concerts.xlsx')
ShowDetails.to_excel(CLEAR_DATA_PATH + 'show_details.xlsx')
ArtistDetails.to_excel(CLEAR_DATA_PATH + 'artist_details.xlsx')
ArtistList.to_excel(CLEAR_DATA_PATH + 'artist_list.xlsx')
ArtistStats.to_excel(CLEAR_DATA_PATH + 'artist_stats.xlsx')
AuthorStats.to_excel(CLEAR_DATA_PATH + 'author_stats.xlsx')
HallsTime.to_excel(CLEAR_DATA_PATH + 'halls_time.xlsx')
FestDays.to_excel(CLEAR_DATA_PATH + 'fest_days.xlsx')

HallList.to_excel(CLEAR_DATA_PATH + 'HallList.xlsx')
